In [ ]:
import time
import numpy as np
import torch
from torch.distributions import Normal
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# tensorboard用
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## 0.目次

1. それぞれの手札の状況から、世界モデルを獲得する部分（RSSM、Dreamer）
2. 共通の盤面の状況から、遷移や価値を求める部分（RNN、Dreamer）
3. 1, 2から行動を求めたり、2と行動から1を予測したりする部分（条件付きVAE）
4. Agentが行動を決定する部分
5. 補助機能の実装

## 1.世界モデルの獲得

演習第5回のRSSM、Dreamerを参考にしつつ、以下を実装する。

* TransitionModel
 * reccurent : 状態遷移
 * prior : 状態遷移を用いた1ステップ先の未来の状態表現の分布
 * posterior : 1ステップ先の観測の情報を取り込んで計算した状態表現の分布
* ObservationModel : 観測を復元するデコーダ
* EncoderModel : 観測から低次元へ



以下のプログラム上の変数と意味
* state (s_t) : priorやposteriorから構成される確率的状態(mean:平均、stddev:標準偏差で再パラメータ化)
* action (a_t) : 行動
* rnn_hidden (h_t) : 状態遷移
* embedded_obs (e_t) : 観測を低次元(64次元)にしたもの
* obs (o_t) : 観測
* hidden : 計算するときの中間層(全て32次元)

In [ ]:
class TransitionModel(nn.Module):
    """
    自分の状況を表す世界モデル
    決定的状態遷移(RNN) : h_t+1 = f(h_t, s_t, a_t)
    確率的状態遷移による1ステップ予測として定義される "prior" : p(s_t+1 | h_t+1)
    観測の情報を取り込んで定義される "posterior": q(s_t+1 | h_t+1, e_t+1)
    """
    def __init__(self, state_dim, action_dim, rnn_hidden_dim, hidden_dim=32, min_stddev=0.1, act=F.elu):
        super(TransitionModel, self).__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.rnn_hidden_dim = rnn_hidden_dim
        self.fc_state_action = nn.Linear(state_dim + action_dim, hidden_dim)
      
        self.fc_rnn_hidden = nn.Linear(rnn_hidden_dim, hidden_dim)
        self.fc_state_mean_prior = nn.Linear(hidden_dim, state_dim)
        self.fc_state_stddev_prior = nn.Linear(hidden_dim, state_dim)

        self.fc_rnn_hidden_embedded_obs = nn.Linear(rnn_hidden_dim + 64, hidden_dim)
        self.fc_state_mean_posterior = nn.Linear(hidden_dim, state_dim)
        self.fc_state_stddev_posterior = nn.Linear(hidden_dim, state_dim)

        #next hidden stateを計算
        self.rnn = nn.GRUCell(hidden_dim, rnn_hidden_dim)
        self._min_stddev = min_stddev
        self.act = act
  

    def forward(self, state, action, rnn_hidden, embedded_next_obs):
        """
        h_t+1 = f(h_t, s_t, a_t)
        prior p(s_t+1 | h_t+1) と posterior q(s_t+1 | h_t+1, e_t+1) を返す
        この2つが近づくように学習する
        """
        next_state_prior, rnn_hidden = self.prior(self.reccurent(state, action, rnn_hidden))
        next_state_posterior = self.posterior(rnn_hidden, embedded_next_obs)
        return next_state_prior, next_state_posterior, rnn_hidden
      
    def reccurent(self, state, action, rnn_hidden):
        """
        h_t+1 = f(h_t, s_t, a_t)を計算する
        """
        hidden = self.act(self.fc_state_action(torch.cat([state, action], dim=1)))
        #h_t+1を求める
        rnn_hidden = self.rnn(hidden, rnn_hidden)
        return rnn_hidden

    def prior(self, rnn_hidden):
        """
        prior p(s_t+1 | h_t+1) を計算する
        """
        #h_t+1を求める
        hidden = self.act(self.fc_rnn_hidden(rnn_hidden))

        mean = self.fc_state_mean_prior(hidden)
        stddev = F.softplus(self.fc_state_stddev_prior(hidden)) + self._min_stddev
        return Normal(mean, stddev), rnn_hidden

    def posterior(self, rnn_hidden, embedded_obs):
        """
        posterior q(s_t+1 | h_t+1, e_t+1)  を計算する
        """
        # h_t+1, o_t+1を結合し, q(s_t+1 | h_t+1, e_t+1) を計算する
        hidden = self.act(self.fc_rnn_hidden_embedded_obs(torch.cat([rnn_hidden, embedded_obs], dim=1)))
        mean = self.fc_state_mean_posterior(hidden)
        stddev = F.softplus(self.fc_state_stddev_posterior(hidden)) + self._min_stddev
        return Normal(mean, stddev)

In [ ]:
class ObservationModel(nn.Module):
    """
    p(o_t | s_t, h_t)
    低次元の状態表現から画像を再構成するデコーダ （23次元: 手札の状態の次元）
    """
    def __init__(self, state_dim, rnn_hidden_dim, hidden_dim=32):
        super(ObservationModel, self).__init__()
        self.fc1 = nn.Linear(state_dim + rnn_hidden_dim, 64)
        self.fc2 = nn.Linear(64, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 23)


    def forward(self, state, rnn_hidden):
        hidden = self.relu(self.fc1(torch.cat([state, rnn_hidden], dim=1)))
        hidden = self.relu(self.fc2(hidden))
        obs = self.fc3(hidden)
        return obs

In [ ]:
class EncoderModel(nn.Module):
    """
    p(e_t | o_t)
    状態を低次元ベクトルに変換するエンコーダ
    """
    def __init__(self, hidden_dim=32):
        super(EncoderModel, self).__init__()
        self.fc1 = nn.Linear(23, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 64)

    def forward(self, obs):
        hidden = F.relu(self.fc1(obs))
        hidden = F.relu(self.fc2(hidden))
        embedded_obs = F.relu(self.fc3(hidden))
        return embedded_obs

上記で定義された3つのモデルを`World`クラスとしてまとめる。

In [ ]:
class World:
    def __init__(self, state_dim, action_dim, rnn_hidden_dim):
        self.transition = TransitionModel(state_dim, action_dim, rnn_hidden_dim).to(device)
        self.observation = ObservationModel(state_dim, rnn_hidden_dim).to(device)
        self.encoder = EncoderModel().to(device)

## 2.盤面の状態遷移

演習第5回のRSSM、Dreamerを参考にしつつ、以下を実装する。

* RnnModel : 状態遷移を行うクラス
* EmbeddingModel : 観測から低次元に変換するクラス
* RewardModel : 報酬を予測するクラス
* (5/23削除)ValueModel : 価値関数を計算するクラス



以下のプログラム上の変数と意味

* state (s_t) : 状態
* action (a_t) : 行動
* rnn_hidden (h_t) : 状態遷移
* obs (o_t) : 観測（盤面の状態）
* hidden : 計算するときの中間層(全て32次元)

In [ ]:
class RnnModel(nn.Module):
    """
    盤面に関する状態遷移
    決定的状態遷移(RNN) : h_t+1 = f(h_t, s_t, a_t)
    次の状態を予測 : p(s_t+1, h_t+1)
    """
    def __init__(self, state_dim, action_dim, rnn_hidden_dim, hidden_dim=32, min_stddev=0.1, act=F.elu):
        super(RnnModel, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, hidden_dim)
        self.state_rnn = nn.GRUCell(hidden_dim, rnn_hidden_dim)
        self.fc2 = nn.Linear(rnn_hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, state_dim)
        self.rnn_hidden_dim = rnn_hidden_dim
        self.act = act
  

    def forward(self, state, action, rnn_hidden):
        # h_t+1を求める
        hidden = self.act(self.fc1(torch.cat([state, action], dim=1)))
        rnn_hidden = self.state_rnn(hidden, rnn_hidden)
        return rnn_hidden
    
    def prior(self, rnn_hidden):
        # s_tを求める
        hidden = self.act(self.fc2(rnn_hidden))
        next_state = self.fc3(hidden)
        return next_state

In [ ]:
class EmbeddingModel(nn.Module):
    """
    p(e_t | o_t)
    状態を低次元ベクトルに変換するエンコーダ
    """
    def __init__(self, hidden_dim=32):
        super(EmbeddingModel, self).__init__()
        self.fc1 = nn.Linear(23, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, state_dim)

    def forward(self, obs):
        hidden = F.relu(self.fc1(obs))
        hidden = F.relu(self.fc2(hidden))
        state = F.relu(self.fc3(hidden))
        return state

In [ ]:
class RewardModel(nn.Module):
    """
    p(r_t | s_t, h_t) 
    低次元の状態表現から報酬を予測する
    """
    def __init__(self, state_dim, rnn_hidden_dim, hidden_dim=32, act=F.elu):
        super(RewardModel, self).__init__()
        self.fc1 = nn.Linear(state_dim + rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)
        self.act = act
 

    def forward(self, state, rnn_hidden, action):
        hidden = self.act(self.fc1(torch.cat([state, rnn_hidden], dim=1)))
        hidden = self.act(self.fc2(hidden))
        hidden = self.act(self.fc3(hidden))
        reward = self.fc4(hidden)
        return reward

上記で定義された3つのモデルを`State`クラスとしてまとめる。

In [ ]:
class State:
    def __init__(self, state_dim, action_dim, rnn_hidden_dim):
        self.rnn = RnnModel(state_dim, action_dim, rnn_hidden_dim).to(device)
        self.embedding = EmbeddingModel().to(device)
        self.reward = RewardModel(state_dim, rnn_hidden_dim).to(device)

## 3.行動と世界モデルの予測

条件付きVAEを参考にしつつ、以下を実装する。
(5/23補足)条件付きVAEは参考にしない。

* ActionModel : 盤面状態と世界モデルから行動を選択する
* DecorderModel : 盤面状態と行動から世界モデルを予測する
* (5/23追加)ValueModel :  盤面状態と世界モデルから価値を予測する

以下のプログラム上の変数と意味

* state: 盤面状態（条件付きVAEのy）
* rnn_hidden: 盤面状態遷移（条件付きVAEのy）
* world : 世界モデル（条件付きVAEのx）
* action : 行動（条件付きVAEのz）
* hidden : 隠れ層(64次元)

※ スライドと、xとzを逆にして定義している。

In [ ]:
class ActionModel(nn.Module):
    """
    世界モデル(world_dim)と低次元の状態表現(state_dim + rnn_hidden_dim)から行動を計算するクラス
    """
    def __init__(self, world_dim, state_dim, rnn_hidden_dim, action_dim,
                 hidden_dim=64, act=F.elu, min_stddev=1e-4, init_stddev=5.0):
        super(ActionModel, self).__init__()
        self.fc1 = nn.Linear(world_dim + state_dim + rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_mean = nn.Linear(hidden_dim, action_dim)
        self.fc_stddev = nn.Linear(hidden_dim, action_dim)
        self.act = act
        self.min_stddev = min_stddev
        self.init_stddev = np.log(np.exp(init_stddev) - 1)

    def forward(self, world, state, rnn_hidden, training=True):
        """
        training=Trueなら, NNのパラメータに関して微分可能な形の行動のサンプル（Reparametrizationによる）を返します
        training=Falseなら, 行動の確率分布の平均値を返します
        """
        hidden = self.act(self.fc1(torch.cat([world, state, rnn_hidden], dim=1)))
        hidden = self.act(self.fc2(hidden))
        hidden = self.act(self.fc3(hidden))

        # Dreamerの実装に合わせて少し平均と分散に対する簡単な変換が入っています
        mean = self.fc_mean(hidden)
        mean = 5.0 * torch.tanh(mean / 5.0)
        stddev = self.fc_stddev(hidden)
        stddev = F.softplus(stddev + self.init_stddev) + self.min_stddev

        if training:
            action = torch.tanh(Normal(mean, stddev).rsample()) # 微分可能にするためrsample()
        else:
            action = torch.tanh(mean)
        # lossの計算で必要なmeanとstddevも追加で返す
        return mean, stddev, action

In [ ]:
class DecoderModel(nn.Module):
    """
    行動の世界モデル(action_dim)と低次元の状態表現(state_dim + rnn_hidden_dim)から世界モデルを計算するクラス
    """
    def __init__(self, world_dim, state_dim, rnn_hidden_dim, action_dim,
                hidden_dim=32, act=F.elu, min_stddev=1e-4, init_stddev=5.0):
        super(DecoderModel, self).__init__()
        self.fc1 = nn.Linear(action_dim + state_dim + rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, world_dim)
 

    def forward(self, state, rnn_hidden, action):
        hidden = self.act(self.fc1(torch.cat([action, state, rnn_hidden], dim=1)))
        hidden = self.act(self.fc2(hidden))
        hidden = self.act(self.fc3(hidden))
        world = self.act(self.fc4(hidden))
        return world

In [ ]:
class ValueModel(nn.Module):
    """
    低次元の状態表現から状態価値を出力する
    """
    def __init__(self, world_dim, state_dim, rnn_hidden_dim, hidden_dim=32, act=F.elu):
        super(ValueModel, self).__init__()
        self.fc1 = nn.Linear(world_dim + state_dim + rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)
        self.act = act

    def forward(self, world, state, rnn_hidden):
        hidden = self.act(self.fc1(torch.cat([world, state, rnn_hidden], dim=1)))
        hidden = self.act(self.fc2(hidden))
        hidden = self.act(self.fc3(hidden))
        state_value = self.fc4(hidden)
        return state_value

上記で定義された3つのモデルを`CVAE`クラスとしてまとめる。

In [ ]:
class CVAE():
    def __init__(self, world_dim, state_dim, rnn_hidden_dim, action_dim):
        self.action = ActionModel(world_dim, state_dim, rnn_hidden_dim, action_dim).to(device)
        self.decoder = DecoderModel(world_dim, state_dim, rnn_hidden_dim, action_dim).to(device)
        self.value = ValueModel(world_dim, state_dim, rnn_hidden_dim).to(device)

## 4.エージェント

第5回演習7章を参考にし、以下を実装する。

* MyAgent : 自分の行動を決定するクラス。手札の状況が入力として与えられ、1、2章で学習した低次元の状態表現を計算し、行動を決定する。
* EnemyAgent : 相手の行動を予測するクラス。今までの状態と行動から相手の世界モデルの状態遷移を元に、1、2章で学習した低次元の状態表現を計算し、行動を決定する。

7章でMyAgentを用いて学習し、8章で学習したモデルをEnemyAgentに適応し検証する。

以下のプログラム上の変数と意味

* world : 1章で作成したクラス
* state : 2章で作成したクラス
* cvae : 3章で作成したクラス
* obs : 盤面の観測
* hand : 自分の手札の観測

In [ ]:
class MyAgent:
    """
    自分の行動を決定するクラス
    """
    def __init__(self, world, state, cvae):
        self.world = world
        self.state = state 
        self.cvae = cvae
        
        self.device = next(cvae.action.parameters()).device
        self.world_rnn_hidden = torch.zeros(1, world.transition.rnn_hidden_dim, device=self.device)
        self.state_rnn_hidden = torch.zeros(1, state.rnn.rnn_hidden_dim, device=self.device)

    def __call__(self, obs, hand, training=True):
        obs = torch.as_tensor(obs, device=self.device)
        hand = torch.as_tensor(hand, device=self.device)

        with torch.no_grad():
            # 手札
            embedded_hand = self.world.encoder(hand)
            state_posterior = self.world.transition.posterior(self.world_rnn_hidden, embedded_hand)
            world = state_posterior.sample()
            # 盤面
            embedded_obs = self.state.embedding(obs)
            state = embedded_obs.sample()
            # 行動選択
            action = self.cvae.action(world, state, self.state_rnn_hidden, training=training)

            # 次のステップのために隠れ状態を更新しておく
            self.world_rnn_hidden = self.world.transition.reccurent(world, action, self.world_rnn_hidden)
            self.state_rnn_hidden = self.state.rnn(state, action, self.state_rnn_hidden)

        return action.squeeze().cpu().numpy()

    #RNNの隠れ状態をリセット
    def reset(self):
        self.world_rnn_hidden = torch.zeros(1, world.transition.rnn_hidden_dim, device=self.device)
        self.state_rnn_hidden = torch.zeros(1, state.rnn.rnn_hidden_dim, device=self.device)

In [ ]:
class EnemyAgent:
    def __init__(self, world, state, cvae):
        self.world = world
        self.state = state
        self.cvae = cvae

        self.device = next(cvae.action.parameters()).device
        self.world_rnn_hidden = torch.zeros(1, world.transition.rnn_hidden_dim, device=self.device)
        self.state_rnn_hidden = torch.zeros(1, state.rnn.rnn_hidden_dim, device=self.device)

    def __call__(self, obs, training=True):
        """
        相手の行動を予測するクラス
        """
        obs = torch.as_tensor(obs, device=self.device)

        with torch.no_grad():
            # 世界モデル
            world = self.world.transition.prior(self.world_rnn_hidden)
            world = world.sample()
            # 盤面
            embedded_obs = self.state.embedding(obs)
            state = embedded_obs.sample()
            # 行動選択
            # でも、何手まで読めばいいのか分からない。
            action = self.cvae.action(world, state, self.state_rnn_hidden, training=training)

            # 次のステップのために隠れ状態を更新しておく
            # predictのworld_rnn_hiddenと同じにしないほうがいい。
            # 相手の行動が実際に行われたら、正しい情報に更新しないといけないから。
            self.world_rnn_hidden = self.world.transition.reccurent(world, action, self.world_rnn_hidden)
            self.state_rnn_hidden = self.state.rnn(state, action, self.state_rnn_hidden)

        return action.squeeze().cpu().numpy()
    
    def predict(self, obs, action):
        """
        相手が行動するたび、相手の世界モデルを更新するために、この関数を実行する
        """
        obs = torch.as_tensor(obs, device=self.device)
        action = torch.as_tensor(action, device=self.device)

        with torch.no_grad():
            # 盤面
            embedded_obs = self.state.embedding(obs)
            state = embedded_obs.sample()
            # 相手の世界モデル
            world = self.cvae.decoder(state, self.state_rnn_hidden, action)

            # 次のステップのために隠れ状態を更新しておく
            self.world_rnn_hidden = self.world.transition.reccurent(world, action, self.world_rnn_hidden)
            self.state_rnn_hidden = self.state.rnn(state, action, self.state_rnn_hidden)


    # RNNの隠れ状態をリセット
    def reset(self):
        self.world_rnn_hidden = torch.zeros(1, world.rnn_hidden_dim, device=self.device)
        self.state_rnn_hidden = torch.zeros(1, state.rnn_hidden_dim, device=self.device)

## 5.補助機能

第5回演習5章を参考にし、以下を実装する。
* リプレイバッファ（myhandsを追加しただけ）
* λ-returnを計算する関数（編集なし）

In [ ]:
# 今回のReplayBuffer
class ReplayBuffer(object):
    """
    RNNを用いて訓練するのに適したリプレイバッファ
    """
    def __init__(self, capacity, observation_shape, hand_shape, action_dim):
        self.capacity = capacity

        self.observations = np.zeros((capacity, *observation_shape), dtype=np.uint8)
        self.myhands = np.zeros((capacity, *hand_shape), dtype=np.uint8)
        # 敵の手札の情報は、世界モデルの学習には使用しない。
        # self.enemyhands = np.zeros((capacity, *hand_shape), dtype=np.uint8)
        self.actions = np.zeros((capacity, action_dim), dtype=np.float32)
        self.rewards = np.zeros((capacity, 1), dtype=np.float32)
        self.done = np.zeros((capacity, 1), dtype=np.bool)

        self.index = 0
        self.is_filled = False

    def push(self, observation, myhand, action, reward, done):
        """
        リプレイバッファに経験を追加する
        """
        self.observations[self.index] = observation
        self.myhands[self.index] = myhand
        self.actions[self.index] = action
        self.rewards[self.index] = reward
        self.done[self.index] = done

        # indexは巡回し, 最も古い経験を上書きする
        if self.index == self.capacity - 1:
            self.is_filled = True
        self.index = (self.index + 1) % self.capacity

    def sample(self, batch_size, chunk_length):
        """
        経験をリプレイバッファからサンプルします. （ほぼ）一様なサンプルです
        結果として返ってくるのは観測(画像), 行動, 報酬, 終了シグナルについての(batch_size, chunk_length, 各要素の次元)の配列です
        各バッチは連続した経験になっています
        注意: chunk_lengthをあまり大きな値にすると問題が発生する場合があります
        """
        episode_borders = np.where(self.done)[0]
        sampled_indexes = []
        for _ in range(batch_size):
            cross_border = True
            while cross_border:
                initial_index = np.random.randint(len(self) - chunk_length + 1)
                final_index = initial_index + chunk_length - 1
                cross_border = np.logical_and(initial_index <= episode_borders,
                                              episode_borders < final_index).any() # 論理積
            sampled_indexes += list(range(initial_index, final_index + 1))

        sampled_observations = self.observations[sampled_indexes].reshape(
            batch_size, chunk_length, *self.observations.shape[1:])
        sampled_myhands = self.myhands[sampled_indexes].reshape(
            batch_size, chunk_length, *self.myhands.shape[1:])
        sampled_actions = self.actions[sampled_indexes].reshape(
            batch_size, chunk_length, self.actions.shape[1])
        sampled_rewards = self.rewards[sampled_indexes].reshape(
            batch_size, chunk_length, 1)
        sampled_done = self.done[sampled_indexes].reshape(
            batch_size, chunk_length, 1)
        return sampled_observations, sampled_myhands, sampled_actions, sampled_rewards, sampled_done

    def __len__(self):
        return self.capacity if self.is_filled else self.index

In [ ]:
def lambda_target(rewards, values, gamma, lambda_):
    """
    価値関数の学習のためのλ-returnを計算します
    """
    V_lambda = torch.zeros_like(rewards, device=rewards.device)

    H = rewards.shape[0] - 1
    V_n = torch.zeros_like(rewards, device=rewards.device)
    V_n[H] = values[H]
    for n in range(1, H+1):
        # まずn-step returnを計算します
        # 注意: 系列が途中で終わってしまったら, 可能な中で最大のnを用いたn-stepを使います
        V_n[:-n] = (gamma ** n) * values[n:]
        for k in range(1, n+1):
            if k == n:
                V_n[:-n] += (gamma ** (n-1)) * rewards[k:]
            else:
                V_n[:-n] += (gamma ** (k-1)) * rewards[k:-n+k]

        # lambda_でn-step returnを重みづけてλ-returnを計算します
        if n == H:
            V_lambda += (lambda_ ** (H-1)) * V_n
        else:
            V_lambda += (1 - lambda_) * (lambda_ ** (n-1)) * V_n

    return V_lambda

## 6.パラメータの設定

第5回演習8章を参考にし、実装する。

In [ ]:
# リプレイバッファの宣言
buffer_capacity = 200000  # Colabのメモリの都合上, 元の実装より小さめにとっています
replay_buffer = ReplayBuffer(capacity=buffer_capacity, 
                             observation_shape=(1, 23),
                             hand_shape=(1, 23),
                             action_dim=3*13)

# モデルの次元設定
world_dim = 16  # 確率的状態（世界モデル）の次元
state_dim = 16  # 盤面状態を圧縮した次元
action_dim = 23  # 行動の次元
rnn_hidden_dim = 64  # 決定的状態（RNNの隠れ状態）の次元
# モデルの宣言
world = World(world_dim, action_dim, rnn_hidden_dim)
state = State(state_dim, action_dim, rnn_hidden_dim)
cvae = CVAE(world_dim, state_dim, rnn_hidden_dim, action_dim)


# 学習率の設定
world_lr = 6e-4
state_lr = 6e-4
cvae_lr = 8e-5
eps = 1e-4
world_params = (list(world.transition.parameters()) +
                list(world.observation.parameters()) +
                list(world.encoder.parameters()))
state_params = (list(state.rnn.parameters()) +
                list(state.embedding.parameters())+
                list(state.reward.parameters()))
world_optimizer = torch.optim.Adam(world_params, lr=world_lr, eps=eps)
state_optimizer = torch.optim.Adam(state_params, lr=state_lr, eps=eps)
action_optimizer = torch.optim.Adam(cvae.action.parameters(), lr=action_lr, eps=eps)
decoder_optimizer = torch.optim.Adam(cvae.decoder.parameters(), lr=action_lr, eps=eps)
value_optimizer = torch.optim.Adam(cvae.value.parameters(), lr=value_lr, eps=eps)

# その他ハイパーパラメータ
# 以下未編集
seed_episodes = 5  # 最初にランダム行動で探索するエピソード数
all_episodes = 100  # 学習全体のエピソード数（300ほどで, ある程度収束します）
test_interval = 10  # 何エピソードごとに探索ノイズなしのテストを行うか
model_save_interval = 20  # NNの重みを何エピソードごとに保存するか
collect_interval = 100  # 何回のNNの更新ごとに経験を集めるか（＝1エピソード経験を集めるごとに何回更新するか）

action_noise_var = 0.3  # 探索ノイズの強さ

batch_size = 50
chunk_length = 50  # 1回の更新で用いる系列の長さ
imagination_horizon = 15  # Actor-Criticの更新のために, Dreamerで何ステップ先までの想像上の軌道を生成するか


gamma = 0.9  # 割引率
lambda_ = 0.95  # λ-returnのパラメータ
clip_grad_norm = 100  # gradient clippingの値
free_nats = 3  # KL誤差（RSSMのTransitionModelにおけるpriorとposteriorの間の誤差）がこの値以下の場合, 無視する

## 7.自分の世界モデルの学習

第5回演習9章を参考にし、実装する。

ここでは自分の世界モデルを獲得し、良い行動選択ができるようになることを目標とする。

### 7.1 Dreamerの学習ループ
```
5回最後まで実行し、経験を貯める。

1エポックごとに1回最後まで実行し、100回NNの更新を行う。

* RSSMの更新
 * リプレイバッファから、バッチサイズ50*系列長さ50 の経験を読み込む
 * 系列長さの50回、次の状態をpriorとposteriorで予測し、klダイバージェンスを計算
 * 観測を再構成し、実際の観測との平均二乗誤差を計算
 * 報酬を予測し、環境から得た報酬との平均二乗誤差を計算
 * 誤差を全て足してRSSMとEncoderの更新

* ActionModel, ValueModelの更新
 * 15ステップ分、確率的状態を入力とする学習可能な行動モデルで、想像上の軌道を作成
 * 架空の軌道に対する報酬と価値を計算し、λ-returnも計算
 * 更新した価値関数で求めた価値が大きくなるように、行動モデルの平均二乗誤差を計算し、行動モデルを更新
 * 価値関数とλ-returnの平均二乗誤差を計算し、価値関数を更新

10エポックごとに探索ノイズなしでテストをし、20エポックごとにモデルを保存する。
```

### 7.2 今回のモデルの学習ループ
```
5回最後まで実行し、経験を貯める。

1エポックごとに1回最後まで実行し、100回NNの更新を行う。

* World（Transition, Observation, Encoder）の更新
 * リプレイバッファから、バッチサイズ50*系列長さ50 の経験を読み込む
 * 系列長さの50回、次の状態をpriorとposteriorで予測し、klダイバージェンスを計算
 * 観測を再構成し、実際の観測との平均二乗誤差を計算
 * 誤差を全て足してWorldの更新

* State（RNN, Enbedding, Reward）の更新
 * リプレイバッファから、バッチサイズ50*系列長さ50 の経験を読み込む
 * RNNで予測した状態と低次元の状態との平均二乗誤差を計算
 * 報酬と環境から得た報酬との平均二乗誤差を計算
 * 誤差を全て足してStateの更新

* CVAE（Action, Decoder, Value）の更新
 * 15ステップ分、確率的状態を入力とする学習可能な行動モデルで、想像上の軌道を作成
 * 架空の軌道に対する報酬と価値を計算し、λ-returnも計算 ？？？
 * 更新した価値関数で求めた価値が大きくなるように、行動モデルの平均二乗誤差を計算し、行動モデルを更新 ？？？
 * 世界モデルを再構成し、実際の世界モデルとの平均二乗誤差を計算
 * 価値関数とλ-returnの平均二乗誤差を計算し、価値関数を更新

10エポックごとに探索ノイズなしでテストをし、20エポックごとにモデルを保存する。
```

### 7.3 問題点

1つ目
```
Dreamerの報酬は、状態のみから報酬がもらえたが、今回の環境は、行動後に行った行動によって報酬がもらえる。
つまり、確率的状態のみから報酬r(s)を求めることは不可能で、報酬を予測するのに、状態と行動を用いるように変更する必要がある。

また、価値関数もv(s)ではなく、行動価値関数q(s,a)を学習しなければいけない。行動が離散的ということも考慮しなければならない。

さらに、ActionModelを求める方法も、λ-returnにマイナスをつけたものではなく、Actor-criticを参考にして、新たに考える必要がある。
（lambda_targetがどういう計算をしているのか、まだ詳しく理解できていない）
```


2つ目
```
CVAEのzは、xを再構成できるという性質をもつものであるので、zには正しい行動を選択できるという性質はない。
そのため、zから行動を選択できるように、さらに層を追加するべきだと思う。

というか、そもそもCVAEを使わず、盤面の状態と世界モデルから、全結合で行動を選択すればいいかも。
```


## 7.4 実装

In [ ]:
# -------------------------------------------------------------------------------------
#  5回最後まで実行し、経験を貯める
# -------------------------------------------------------------------------------------
env = make_env()
for episode in range(seed_episodes):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = decision_steps.agent_id[0]
    done = False

    while not done:
        # 環境から観測を入手
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        obs = decision_steps.obs
        myhand = decision_steps.obs  # <=================ここ変更

        # ランダムに行動選択
        action = spec.action_spec.random_action(len(decision_steps))
        # 環境の更新
        env.set_actions(behavior_name, action)
        env.step()

        # 環境から終了かどうかの判別
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        if tracked_agent in decision_steps:
            reward = decision_steps[tracked_agent].reward
            episode_reward += reward
        if tracked_agent in terminal_steps:
            reward = terminal_steps[tracked_agent].reward
            episode_reward += reward
            done=True
        
        # 記録
        replay_buffer.push(obs, myhand, action, reward, done)

In [ ]:
env = make_env()
for episode in range(seed_episodes, all_episodes):
    start = time.time()

    # -------------------------------------------------------------------------------------
    #  1エポックごとに1回最後まで実行
    # -------------------------------------------------------------------------------------
    # 行動を決定するためのエージェントを宣言
    policy = MyAgent(world, state, cvae)  # まだ
    # 環境のリセット
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = decision_steps.agent_id[0]
    done = False
    episode_reward = 0
    while not done:
        # 環境から観測を入手
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        obs = decision_steps.obs
        myhand = decision_steps.obs  # <=================ここ変更

        # 行動選択をし、ノイズを加える。
        action = policy(obs, myhand)
        action += np.random.normal(0, np.sqrt(action_noise_var), action.shape)
        # 環境の更新
        env.set_actions(behavior_name, action)
        env.step()
        
        # 環境から終了かどうかの判別
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        if tracked_agent in decision_steps:
            reward = decision_steps[tracked_agent].reward
            episode_reward += reward
        if tracked_agent in terminal_steps:
            reward = terminal_steps[tracked_agent].reward
            episode_reward += reward
            done=True

        # 記録
        replay_buffer.push(obs, myhand, action, reward, done)

    # 訓練時の報酬と経過時間をログとして表示
    writer.add_scalar('total reward at train', episode_reward, episode)
    print('episode [%4d/%4d] is collected. Total reward is %f' %
            (episode+1, all_episodes, episode_reward))
    print('elasped time for interaction: %.2fs' % (time.time() - start))


    # -------------------------------------------------------------------------------------
    #  100回NNの更新を行う
    # -------------------------------------------------------------------------------------
    start = time.time()
    for update_step in range(collect_interval):
        # -------------------------------------------------------------------------------------
        #  World（Transition, Observation, Encoder）の更新
        # -------------------------------------------------------------------------------------
        # リプレイバッファから、バッチサイズ50*系列長さ50 の経験を読み込む
        observations, myhands, actions, rewards, _ = replay_buffer.sample(batch_size, chunk_length)

        # 観測の前処理
        observations = torch.as_tensor(obs, device=device)
        myhands = torch.as_tensor(myhands, device=device)
        actions = torch.as_tensor(actions, device=device)
        rewards = torch.as_tensor(rewards, device=device)

        # 観測をエンコーダで低次元のベクトルに変換
        embedded_hand = world.encoder(myhands).view(chunk_length, batch_size, -1)

        # 低次元の状態表現を保持しておくためのTensorを定義
        worlds = torch.zeros(chunk_length, batch_size, world_dim, device=device)
        world_rnn_hiddens = torch.zeros(chunk_length, batch_size, rnn_hidden_dim, device=device)
        # 低次元の状態表現は最初はゼロ初期化
        world = torch.zeros(batch_size, world_dim, device=device)
        world_rnn_hidden = torch.zeros(batch_size, rnn_hidden_dim, device=device)

        # 系列長さの50回、次の状態をpriorとposteriorで予測し、klダイバージェンスを計算
        kl_loss = 0
        for l in range(chunk_length-1):
            next_state_prior, next_state_posterior, world_rnn_hidden = \
                world.transition(world, actions[l], world_rnn_hidden, embedded_hand[l+1])
            world = next_state_posterior.rsample()
            worlds[l+1] = world
            world_rnn_hiddens[l+1] = world_rnn_hidden
            kl = kl_divergence(next_state_prior, next_state_posterior).sum(dim=1)
            kl_loss += kl.clamp(min=free_nats).mean()  # 原論文通り, KL誤差がfree_nats以下の時は無視
        kl_loss /= (chunk_length - 1)

        # states[0] and rnn_hiddens[0]はゼロ初期化なので以降では使わない
        worlds = worlds[1:]
        world_rnn_hiddens = world_rnn_hiddens[1:]

        # 観測を再構成し、実際の観測との平均二乗誤差を計算
        flatten_worlds = worlds.view(-1, world_dim)
        flatten_world_rnn_hiddens = world_rnn_hiddens.view(-1, rnn_hidden_dim)
        recon_myhands = world.observation(flatten_worlds, flatten_world_rnn_hiddens).view(chunk_length-1, batch_size, 3, 23)
        hand_loss = 0.5 * F.mse_loss(recon_myhands, myhands[:-1], reduction='none').mean([0, 1]).sum()

        # 誤差を全て足してWorldの更新
        world_loss = kl_loss + hand_loss
        world_optimizer.zero_grad()
        world_loss.backward()
        clip_grad_norm_(world_params, clip_grad_norm)
        world_optimizer.step()

        # 勾配の流れを遮断
        flatten_worlds = flatten_worlds.detach()
        flatten_world_rnn_hiddens = flatten_world_rnn_hiddens.detach()

        # -------------------------------------------------------------------------------------
        #  State（RNN, Enbedding）の更新
        # -------------------------------------------------------------------------------------

        # 観測をエンコーダで低次元のベクトルに変換
        embedded_obs = state.embedding(observations).view(chunk_length, batch_size, -1)

        # 低次元の状態表現を保持しておくためのTensorを定義
        states = torch.zeros(chunk_length, batch_size, state_dim, device=device)
        state_rnn_hiddens = torch.zeros(chunk_length, batch_size, rnn_hidden_dim, device=device)
        # 低次元の状態表現は最初はゼロ初期化
        state = torch.zeros(batch_size, state_dim, device=device)
        state_rnn_hidden = torch.zeros(batch_size, rnn_hidden_dim, device=device)

        # RNNを実行
        for l in range(chunk_length-1):
            state_rnn_hidden = state.rnn(state, actions[l], state_rnn_hidden)
            next_state = state.rnn.prior(state_rnn_hidden)
            state = embedded_obs[l]  # 盤面の低次元状態は観測を圧縮したもの
            states[l+1] = state
            state_rnn_hiddens[l+1] = state_rnn_hidden
        
        # states[0] and rnn_hiddens[0]はゼロ初期化なので以降では使わない
        states = states[1:]
        state_rnn_hiddens = state_rnn_hiddens[1:]

        flatten_states = states.view(-1, state_dim)
        flatten_state_rnn_hiddens = state_rnn_hiddens.view(-1, rnn_hidden_dim)
        flatten_actions = actions[:-1].view(-1, action_dim)

        # RNNで予測した状態と低次元の状態との平均二乗誤差を計算
        recon_observations = state.rnn.prior(flatten_state_rnn_hiddens).view(chunk_length-1, batch_size, 3, 23)
        obs_loss = 0.5 * F.mse_loss(recon_observations, flatten_states)
  
        # 報酬を予測し、環境から得た報酬との平均二乗誤差を計算
        predicted_rewards = state.reward(flatten_states, flatten_state_rnn_hiddens).view(chunk_length-1, batch_size, 1)
        reward_loss = 0.5 * F.mse_loss(predicted_rewards, rewards[:-1])

        # 勾配降下で更新する
        state_loss = obs_loss + reward_loss
        state_optimizer.zero_grad()
        state_loss.backward()
        clip_grad_norm_(state_params, clip_grad_norm)
        state_optimizer.step()

        # 勾配の流れを遮断
        flatten_states = flatten_states.detach()
        flatten_state_rnn_hiddens = flatten_state_rnn_hiddens.detach()

        # -------------------------------------------------------------------------------------
        #  CVAE（Action, Decoder, Value）の更新
        # -------------------------------------------------------------------------------------

        # DreamerにおけるActor-Criticの更新のために, 現在のモデルを用いた数ステップ先の未来の状態予測を保持するためのTensorを用意
        imaginated_worlds = torch.zeros(imagination_horizon + 1, *flatten_worlds.shape, device=flatten_worlds.device)
        imaginated_world_rnn_hiddens = torch.zeros(imagination_horizon + 1, *flatten_world_rnn_hiddens.shape, device=flatten_world_rnn_hiddens.device)
        imaginated_states = torch.zeros(imagination_horizon + 1, *flatten_states.shape, device=flatten_states.device)
        imaginated_state_rnn_hiddens = torch.zeros(imagination_horizon + 1, *flatten_state_rnn_hiddens.shape, device=flatten_state_rnn_hiddens.device)
        imaginated_actions = torch.zeros(imagination_horizon + 1, *flatten_actions.shape, device=flatten_actions.device)

        # 未来予測をして想像上の軌道を作る前に, 最初の状態としては先ほどモデルの更新で使っていた
        # リプレイバッファからサンプルされた観測データを取り込んだ上で推論した状態表現を使う
        imaginated_worlds[0] = flatten_worlds
        imaginated_world_rnn_hiddens[0] = flatten_world_rnn_hiddens
        imaginated_states[0] = flatten_states
        imaginated_state_rnn_hiddens[0] = flatten_state_rnn_hiddens
        imaginated_actions[0] = flatten_actions
        
        # open-loopで未来の状態予測を使い, 想像上の軌道を作る
        for h in range(1, imagination_horizon + 1):
            # 行動はActionModelで決定. この行動はモデルのパラメータに対して微分可能で, これを介してActionModelは更新される
            actions = cvae.action(flatten_worlds, flatten_states, flatten_state_rnn_hiddens)
            flatten_world_prior, flatten_world_rnn_hiddens = \
                world.transition.prior(world.transition.reccurent(flatten_worlds, actions, flatten_world_rnn_hiddens))
            flatten_state_rnn_hiddens = state.rnn(flatten_states, actions, flatten_state_rnn_hiddens)
            flatten_state_prior = state.rnn.prior(flatten_state_rnn_hiddens)

            flatten_worlds = flatten_world_prior.rsample()
            flatten_states = flatten_state_prior.rsample()
            imaginated_worlds[h] = flatten_worlds
            imaginated_world_rnn_hiddens[h] = flatten_world_rnn_hiddens
            imaginated_states[h] = flatten_states
            imaginated_state_rnn_hiddens[h] = flatten_state_rnn_hiddens
            imaginated_actions[h] = actions

        # RSSMのreward_modelにより予測された架空の軌道に対する報酬を計算
        flatten_imaginated_worlds = imaginated_worlds.view(-1, world_dim)
        flatten_imaginated_world_rnn_hiddens = imaginated_world_rnn_hiddens.view(-1, rnn_hidden_dim)
        flatten_imaginated_states = imaginated_states.view(-1, state_dim)
        flatten_imaginated_state_rnn_hiddens = imaginated_state_rnn_hiddens.view(-1, rnn_hidden_dim)
        flatten_imaginated_actions = imaginated_actions.view(-1, action_dim)
        imaginated_rewards = state.reward(flatten_imaginated_states, flatten_imaginated_state_rnn_hiddens).view(imagination_horizon + 1, -1)
        imaginated_values = cvae.value(flatten_imaginated_worlds, flatten_imaginated_states, flatten_imaginated_state_rnn_hiddens).view(imagination_horizon + 1, -1)

        # λ-returnのターゲットを計算(V_{\lambda}(s_{\tau})
        lambda_target_values = lambda_target(imaginated_rewards, imaginated_values, gamma, lambda_)
        
        # 価値関数の予測した価値が大きくなるようにActionModelを更新. PyTorchの基本は勾配降下だが, 今回は大きくしたいので-1をかける
        action_loss = -lambda_target_values.mean()
        action_optimizer.zero_grad()
        action_loss.backward()
        clip_grad_norm_(cvae.action.parameters(), clip_grad_norm)
        action_optimizer.step()

        # 世界モデルを再構成し、実際の世界モデルとの平均二乗誤差を計算
        decoder_world = cvae.decoder(flatten_imaginated_states.detach(), flatten_imaginated_state_rnn_hiddens.detach(), flatten_imaginated_actions.detach())
        decoder_loss = -lambda_target_values.mean()
        decoder_optimizer.zero_grad()
        decoder.backward()
        clip_grad_norm_(cvae.decoder.parameters(), clip_grad_norm)
        decoder_optimizer.step()

        # TD(λ)ベースの目的関数で価値関数を更新（価値関数のみを学習するため，学習しない変数のグラフは切っている. )
        imaginated_values = cvae.value(flatten_imaginated_worlds.detach(), flatten_imaginated_states.detach(), flatten_imaginated_state_rnn_hiddens.detach()).view(imagination_horizon + 1, -1)        
        value_loss = 0.5 * F.mse_loss(imaginated_values, lambda_target_values.detach())
        value_optimizer.zero_grad()
        value_loss.backward()
        clip_grad_norm_(cvae.value.parameters(), clip_grad_norm)
        value_optimizer.step()

        # ログをTensorBoardに出力
        print('update_step: %3d world_loss: %.5f (kl_loss: %.5f + hand_loss: %.5f),'
              'state_loss: %.5f (obs_loss: %.5f + reward_loss: %.5f),'
              'action_loss: %.5f, decoder_loss: %.5f value_loss: %.5f'
            % (update_step + 1, world_loss.item(), kl_loss.item(), hand_loss.item(), 
               state_loss.item(), obs_loss.item(), reward_loss.item(), action_loss.item(), decoder_loss.item(), value_loss.item())))
        
        total_update_step = episode * collect_interval + update_step
        writer.add_scalar('world loss', world_loss.item(), total_update_step)
        writer.add_scalar('kl loss', kl_loss.item(), total_update_step)
        writer.add_scalar('hand loss', hand_loss.item(), total_update_step)
        writer.add_scalar('state loss', state_loss.item(), total_update_step)
        writer.add_scalar('obs loss', obs_loss.item(), total_update_step)
        writer.add_scalar('reward loss', reward_loss.item(), total_update_step)
        writer.add_scalar('action loss', action_loss.item(), total_update_step)
        writer.add_scalar('decoder loss', decoder_loss.item(), total_update_step)
        writer.add_scalar('value loss', value_loss.item(), total_update_step)

    print('elasped time for update: %.2fs' % (time.time() - start))


    # -------------------------------------------------------------------------------------
    #  10エポックごとに探索ノイズなしでテスト
    # -------------------------------------------------------------------------------------
    """
    if (episode + 1) % test_interval == 0:
        policy = Agent(encoder, rssm.transition, action_model)
        start = time.time()
        obs = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = policy(obs, training=False)
            obs, reward, done, _ = env.step(action)
            total_reward += reward

        writer.add_scalar('total reward at test', total_reward, episode)
        print('Total test reward at episode [%4d/%4d] is %f' %
                (episode+1, all_episodes, total_reward))
        print('elasped time for test: %.2fs' % (time.time() - start))
    """
    # -------------------------------------------------------------------------------------
    #  20エポックごとにモデルを保存
    # -------------------------------------------------------------------------------------
    """
    if (episode + 1) % model_save_interval == 0:
        model_log_dir = os.path.join(log_dir, 'episode_%04d' % (episode + 1))
        os.makedirs(model_log_dir)
        torch.save(encoder.state_dict(), os.path.join(model_log_dir, 'encoder.pth'))
        torch.save(rssm.transition.state_dict(), os.path.join(model_log_dir, 'rssm.pth'))
        torch.save(rssm.observation.state_dict(), os.path.join(model_log_dir, 'obs_model.pth'))
        torch.save(rssm.reward.state_dict(), os.path.join(model_log_dir, 'reward_model.pth'))
        torch.save(value_model.state_dict(), os.path.join(model_log_dir, 'value_model.pth'))
        torch.save(action_model.state_dict(), os.path.join(model_log_dir, 'action_model.pth'))
    """

## 8.相手の世界モデルを予測して対戦